In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import random
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
import seaborn as sns

In [ ]:
# Function to preprocess images
def preprocess_images(directory, condition, target_size):
    images = []
    labels = []
    for filename in os.listdir(directory):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(directory, filename)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert image to RGB format
            img = cv2.resize(img, target_size)
            images.append(img)
            labels.append(condition)
    return images, labels

In [ ]:
# Specify the directories for each condition
normal_dir = 'Path to Normal file'
atelectasis_dir = 'Path to Atelectasis file'
pneumonia_dir = 'Path to Pneumonia file'

# Specify the target size for resizing
target_size = (224, 224)  # DenseNet input size

In [ ]:
# Preprocess images and extract features for each condition
normal_images, normal_labels = preprocess_images(normal_dir, 'Normal', target_size)
atelectasis_images, atelectasis_labels = preprocess_images(atelectasis_dir, 'Atelectasis', target_size)
pneumonia_images, pneumonia_labels = preprocess_images(pneumonia_dir, 'Pneumonia', target_size)

# Combine data from all conditions
X = np.array(normal_images + atelectasis_images + pneumonia_images)
y = np.array(normal_labels + atelectasis_labels + pneumonia_labels)

In [ ]:
# Map string labels to integer labels
label_mapping = {'Normal': 0, 'Atelectasis': 1, 'Pneumonia': 2}
y = np.array([label_mapping[label] for label in y])

# Convert class labels to one-hot encoded format
y = tf.keras.utils.to_categorical(y, num_classes=3)

# Splitting data into training and testing sets (70:30 ratio)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
# Normalize pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0

# Load pre-trained DenseNet121 model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(target_size[0], target_size[1], 3))

# Add GlobalAveragePooling2D layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

In [ ]:
# Add Dense layers
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

# Combine base model and top layers
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
y_pred = np.argmax(model.predict(X_test), axis=-1)
y_true = np.argmax(y_test, axis=-1)
accuracy = accuracy_score(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred)
class_report = classification_report(y_true, y_pred, target_names=label_mapping.keys())

In [ ]:
print("Accuracy:", accuracy)

In [ ]:
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
# Manually input the confusion matrix values
# Replace these values with your actual confusion matrix values
conf_matrix_values = np.array('Fill the confusion matrix values printed here')

# Labels for the confusion matrix
labels = ['Normal', 'Atelectasis', 'Pneumonia']

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_values, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels,
            yticklabels=labels,
            annot_kws={"size": 16})  # Set font size for annotations
plt.title('Confusion Matrix', fontsize=18)  # Set title font size
plt.xlabel('Predicted Label', fontsize=14)  # Set x-axis label font size
plt.ylabel('True Label', fontsize=14)  # Set y-axis label font size
plt.show()

In [ ]:
print("Classification Report:\n", class_report)

In [ ]:
# Display random images with predicted and true outcomes in a 5x5 matrix
num_images = 10
plt.figure(figsize=(15, 15))  # Define the size of the figure

for i in range(num_images):
    index = random.randint(0, len(X_test) - 1)
    img = X_test[index]  # Get the image
    true_label = list(label_mapping.keys())[list(label_mapping.values()).index(np.argmax(y_test[index]))]  # Get the true label
    pred_label = list(label_mapping.keys())[y_pred[index]]  # Get the predicted label

    plt.subplot(5, 5, i+1)
    plt.imshow(img)
    plt.title(f'True Outcome: {true_label}\nPredicted Outcome: {pred_label}')
    plt.axis('off')

plt.tight_layout()
plt.show()